In [1]:
import os
import requests
from dotenv import load_dotenv
import base64
import json
import webbrowser
import urllib
import tqdm
import pandas as pd
from datetime import datetime, date

In [15]:
load_dotenv()
REDIRECT_URI = "http://localhost:8501/"
CLIENT_ID = os.getenv("client_id")
CLIENT_SECRET = os.getenv("client_secret")
SCOPE = " ".join(['user-library-read', 'user-top-read'])
SCOPE_USER_TOP_READ = 'user-top-read'

## Request User Authorization

In [16]:
# Create the authorization URL
params = {
    'response_type': 'code',
    'client_id': CLIENT_ID,
    'scope': SCOPE,
    'redirect_uri': REDIRECT_URI,
}
url = f"https://accounts.spotify.com/authorize?{urllib.parse.urlencode(params)}"

# Open the authorization URL in the default web browser
webbrowser.open(url)

True

In [17]:
CODE = "AQBQ2jj2aWuYVCN0wc3RWEMLdnOdFtcLmZwqqm4kp4txfUO3hXbz8Sidh42QihxEe6VDPqIrcEoIPfaif9VMD53WvdkehIwihiwc-2gj98XEPU5hdQI-HNZq0a-GVrv4fGKdTgYJaYQ-mi0e1XN4usYliaNTKSTDyH6uKQNRJr_tIA26KWShWdq21tOMyvrZm_YXpG1dTuQtp-6E"

## Get Token Info
- Lasts only 1 hour

In [18]:
def get_token_info():
    auth_header =base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    url ="https://accounts.spotify.com/api/token"
    headers={
            "Authorization": f"Basic {auth_header}",
            "Content-Type": "application/x-www-form-urlencoded",}
    data={
            "grant_type": "authorization_code",
            "code" : CODE,
            "redirect_uri": REDIRECT_URI}
    result=requests.post(url, headers=headers, data=data)
    json_results=json.loads(result.content)
    return json_results

token_details=get_token_info()

In [19]:
token_details

{'access_token': 'BQAsZkmgYlCrBICO60ShpwLeaUrl25OuOYAlJeg-zuyRu1jkTYHRjytp_0vYoGXUIPQWBiWspldbRFobqFqAs0X7BU_HBrD7C4C3xbnvwPfZcMaMhx78l-y5_eKa7Hw_UQU22Uq2rM1VO3Mn6eg07Bzj_Db_JvlD7ikIlXVgX56pGFx4UbZgVVpXAgI7Hp47zX5N',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQBlXInLse2SUsoBs4Hrj_EXv2LvJqHAFbmJ-BNJeksywxTEkt_2BK4llX4ZsJCPEiOSJUgKOMnIw6K1Z8BUUt55uO4ZOrjawZNzOEKmkJSG18GFkwSWcV69u4dyd2WT2-w',
 'scope': 'user-library-read user-top-read'}

In [21]:
token = token_details["access_token"]
refresh_token = token_details["refresh_token"]
scope = token_details["scope"]

## Get Auth Header

In [22]:
def get_auth_header(token):
    return f"Bearer " + token

## Post Request
- Request using token for API endpoints

In [11]:
SPOTIFY_GET_CURRENT_USER_SAVED_TRACKS_URL = "https://api.spotify.com/v1/me/tracks?limit=50&offset=0"

In [12]:
def get_current_user_tracks(url):
    response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json

In [13]:
def loop_through():
    first_spotify_url = "https://api.spotify.com/v1/me/tracks?limit=50&offset=0"
    results = get_current_user_tracks(first_spotify_url)
    full_list = []
    current_url = results["href"]
    while results["next"] is not None:
        results = get_current_user_tracks(current_url)
        full_list.append(results)
        spotify_url_next = results["next"]
        current_url = spotify_url_next
    return full_list
            

In [14]:
saved_tracks_data = loop_through()

## Results

In [61]:
saved_tracks_parsed={}
for index_i, i in enumerate(saved_tracks_data):  
    for index_j, j in enumerate(saved_tracks_data[index_i]["items"]):
        saved_tracks_parsed[j["track"]["uri"]] = {
              "added_to_playlist_time":j["added_at"]
            , "artist_name":j["track"]["album"]["artists"][0]["name"]
            , "artist_image":j['track']['album']['images'][1]['url']
            , "unique_artist_id": j["track"]["album"]["artists"][0]["uri"]
            , "release_date": j["track"]["album"]["release_date"]
            , "song_name": j["track"]["name"]
            , "song_length_seconds": int(j["track"]["duration_ms"])/1000
            , "song_explicit": str(j["track"]["explicit"])
            , "song_linkId": j["track"]["external_urls"]["spotify"]
            , "song_popularity": int(j["track"]["popularity"])
            , "album_name": j["track"]["album"]["name"]
            , "spotify_uri_album": j["track"]["album"]["uri"]
            , "spotify_track_id": j["track"]["id"]  
                                            }

In [94]:
saved_tracks_parsed["items"][0]

KeyError: 'items'

## Get song genres from Get Track API Endpoint
### Seems as if most songs do not have a genre object/attribute associated with them (shelving this for now)

In [15]:
# for i in full_list_parsed[0:10]:
#     print(i[0])

# full_list_parsed[0]["0.0"][-1].split(":")[-1]
# list(full_list_parsed[0].keys())

In [16]:
# def get_track_genres(url):
#     response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
#     resp_json = response.json()
#     return resp_json


# get_track_genres("https://api.spotify.com/v1/tracks?ids=6POZ7ocsOQQvJrfArz6KOD")
# for track_id in full_list_parsed:
#     get_track_genre_url= f"https://api.spotify.com/v1/tracks/{track_id}"
#     get_track_genres(get_track_genre_url)

{'tracks': [{'album': {'album_type': 'single',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/757FXqX0Osk2pqtgv4E5v4'},
      'href': 'https://api.spotify.com/v1/artists/757FXqX0Osk2pqtgv4E5v4',
      'id': '757FXqX0Osk2pqtgv4E5v4',
      'name': 'NURKO',
      'type': 'artist',
      'uri': 'spotify:artist:757FXqX0Osk2pqtgv4E5v4'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/0p3tzEAt0XWrBqbrwBoN1I'},
      'href': 'https://api.spotify.com/v1/artists/0p3tzEAt0XWrBqbrwBoN1I',
      'id': '0p3tzEAt0XWrBqbrwBoN1I',
      'name': 'Kyle Hume',
      'type': 'artist',
      'uri': 'spotify:artist:0p3tzEAt0XWrBqbrwBoN1I'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
 

## Call Track Auto Features API endpoint
- https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features

In [17]:
def get_tracks_audio_analysis(url):
    response = requests.get(url=url, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json

### Creating string for audio track inputs
- Spotify limits the inputs to 100 track ids at a time

In [18]:
listofids=[]
for index, i in enumerate(list(saved_tracks_parsed.values())):
    listofids.append(i["spotify_track_id"])

In [19]:
len(listofids)/100

45.06

## Breaking up Id Lists
- Track Audio Analysis only takes in 100 ids at a time

In [20]:
def list_chunks(l,n):
    for i in range(0,len(l), n):
        yield l[i:i + n]

n = 100

chunksoftrackids=list(list_chunks(listofids,n))

In [21]:
len(chunksoftrackids)

46

In [22]:
string_urls=[]
for index,chunk in enumerate(chunksoftrackids):
    string_urls.append(','.join(chunk))

## Getting Track Audio Features

In [23]:
track_audio_features={}
for index,i in enumerate(string_urls):
    audiofeatures= get_tracks_audio_analysis(f'https://api.spotify.com/v1/audio-features?ids={string_urls[index]}')
    for j in audiofeatures["audio_features"]:
        track_audio_features[j['uri']] ={
                "danceability":j["danceability"]
                , "energy":j["energy"]
                , "key":j["key"]
                , "loudness":j["loudness"]
                , "mode":j["mode"]
                , "speechiness":j["speechiness"]
                , "acousticness":j["acousticness"]
                , "instrumentalness":j["instrumentalness"]
                , "liveness":j["liveness"]
                , "valence":j["valence"]
                , "time_signature":j["time_signature"]
                , "tempo":j["tempo"]
                                        }

In [24]:
track_details_all = {k: {**saved_tracks_parsed[k], **track_audio_features[k]} for k in saved_tracks_parsed}

In [25]:
len(track_details_all)

4506

In [26]:
# @dataclass
# class Song:
#     song_id: str
#     added_to_playlist_time: datetime
#     artist_name: str
#     unique_artist_id: str
#     release_date: date
#     song_name: str
#     song_length_seconds: int
#     song_explicit: bool
#     song_linkId: str
#     song_popularity: int
#     album_name: str
#     spotify_uri_album: str
#     spotify_track_id: str
#     danceability: float
#     energy: float
#     key: int
#     loudness: float
#     mode: float
#     speechiness: float
#     acousticness: float
#     instrumentalness: float
#     liveness: float
#     valence: float
#     time_signature: int
#     tempo: float

In [27]:
# dataclass_song_data =[Song(k, **v) for k,v in track_details_all.items()]

In [28]:
temp=list(track_details_all.values())

In [29]:
track_details_df=pd.DataFrame(temp)

In [40]:
track_details_df.head()

,added_to_playlist_time,artist_name,artist_image,unique_artist_id,release_date,song_name,song_length_seconds,song_explicit,song_linkId,song_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,time_signature,tempo
0,2023-09-22T19:11:14Z,Doja Cat,https://i.scdn.co/image/ab67616d00001e027acee9...,spotify:artist:5cj0lLjcoR7YOSnhnX0Po5,2023-08-04,Paint The Town Red,231.750,True,https://open.spotify.com/track/2IGMVunIBsBLtEQ...,100,...,5,-8.603,1,0.1740,0.26900,0.000003,0.0901,0.732,4,99.968
1,2023-09-02T01:35:57Z,Grace Grundy,https://i.scdn.co/image/ab67616d00001e02763ea6...,spotify:artist:3pFPBG84k5bQVbL9apf2PD,2016-11-25,(Everything I Do) I Do It for You,123.283,False,https://open.spotify.com/track/4Sope1x1EUQiWPy...,49,...,0,-8.686,1,0.0368,0.89600,0.000000,0.0961,0.243,4,128.841
2,2023-08-23T20:03:00Z,Jeremy Zucker,https://i.scdn.co/image/ab67616d00001e024f491a...,spotify:artist:3gIRvgZssIb9aiirIg0nI3,2023-06-09,i need you (in my life),189.106,True,https://open.spotify.com/track/5NMl4DvH391bWzv...,49,...,8,-7.650,1,0.0307,0.10100,0.000062,0.1160,0.625,4,130.015
3,2023-08-21T23:04:49Z,Parannoul,https://i.scdn.co/image/ab67616d00001e028ef3be...,spotify:artist:7eZbNxarrTW4VkRI8u9aDX,2023-01-28,Polaris,262.500,False,https://open.spotify.com/track/6Yt8PGw2VEbLH5p...,36,...,6,-5.812,1,0.0351,0.00259,0.001750,0.1200,0.158,4,80.015
4,2023-08-21T23:04:07Z,Caroline Polachek,https://i.scdn.co/image/ab67616d00001e02de1789...,spotify:artist:4Ge8xMJNwt6EEXOzVXju9a,2023-02-14,Billions,297.514,False,https://open.spotify.com/track/4JPF8Te7Jo3AuI2...,50,...,5,-7.892,1,0.1090,0.16900,0.000067,0.0994,0.361,4,139.939


In [30]:
track_details_df.to_csv('song_data.csv', sep=',', index=False)

In [31]:
# track_details_df.head()

In [32]:
# corr = track_details_df.select_dtypes(include=[np.int64, float]).corr()
# Create a heatmap
# plt.figure(figsize=(10,8)) 
# sns.heatmap(corr.round(2), annot=True, cmap='coolwarm')

In [33]:
# print(track_details_df.columns)
# track_details_df.groupby(["artist_name"])["artist_name"].count().sort_values(ascending=False)[0:11]
# This only counts them if they are the main artist

In [34]:
# (track_details_df["song_explicit"].value_counts()/len(track_details_df))*100

In [35]:
# track_details_df["added_to_playlist_month"] = pd.to_datetime(track_details_df.added_to_playlist_time).dt.strftime('%Y-%m')

In [36]:
# plt.figure(figsize=(25, 10))
# plt.bar(data=track_details_df.groupby("added_to_playlist_month").count().sort_values(by=["added_to_playlist_month"])["song_name"], x=track_details_df.groupby("added_to_playlist_month").count().sort_values(by=["added_to_playlist_month"])["song_name"].index, height=track_details_df.groupby("added_to_playlist_month").count().sort_values(by=["added_to_playlist_month"])["song_name"].values)

In [37]:
# temp = pd.DataFrame(pd.read_csv('song_data.csv'))
# temp.head()

,added_to_playlist_time,artist_name,artist_image,unique_artist_id,release_date,song_name,song_length_seconds,song_explicit,song_linkId,song_popularity,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,time_signature,tempo
0,2023-09-22T19:11:14Z,Doja Cat,https://i.scdn.co/image/ab67616d00001e027acee9...,spotify:artist:5cj0lLjcoR7YOSnhnX0Po5,2023-08-04,Paint The Town Red,231.750,True,https://open.spotify.com/track/2IGMVunIBsBLtEQ...,100,...,5,-8.603,1,0.1740,0.26900,0.000003,0.0901,0.732,4,99.968
1,2023-09-02T01:35:57Z,Grace Grundy,https://i.scdn.co/image/ab67616d00001e02763ea6...,spotify:artist:3pFPBG84k5bQVbL9apf2PD,2016-11-25,(Everything I Do) I Do It for You,123.283,False,https://open.spotify.com/track/4Sope1x1EUQiWPy...,49,...,0,-8.686,1,0.0368,0.89600,0.000000,0.0961,0.243,4,128.841
2,2023-08-23T20:03:00Z,Jeremy Zucker,https://i.scdn.co/image/ab67616d00001e024f491a...,spotify:artist:3gIRvgZssIb9aiirIg0nI3,2023-06-09,i need you (in my life),189.106,True,https://open.spotify.com/track/5NMl4DvH391bWzv...,49,...,8,-7.650,1,0.0307,0.10100,0.000062,0.1160,0.625,4,130.015
3,2023-08-21T23:04:49Z,Parannoul,https://i.scdn.co/image/ab67616d00001e028ef3be...,spotify:artist:7eZbNxarrTW4VkRI8u9aDX,2023-01-28,Polaris,262.500,False,https://open.spotify.com/track/6Yt8PGw2VEbLH5p...,36,...,6,-5.812,1,0.0351,0.00259,0.001750,0.1200,0.158,4,80.015
4,2023-08-21T23:04:07Z,Caroline Polachek,https://i.scdn.co/image/ab67616d00001e02de1789...,spotify:artist:4Ge8xMJNwt6EEXOzVXju9a,2023-02-14,Billions,297.514,False,https://open.spotify.com/track/4JPF8Te7Jo3AuI2...,50,...,5,-7.892,1,0.1090,0.16900,0.000067,0.0994,0.361,4,139.939


## Getting Most associated Artists and Tracks

### Request user Auth

In [30]:
# Create the authorization URL
params = {
    'response_type': 'code',
    'client_id': CLIENT_ID,
    'scope': SCOPE_USER_TOP_READ,
    'redirect_uri': REDIRECT_URI,
}
url = f"https://accounts.spotify.com/authorize?{urllib.parse.urlencode(params)}"

# Open the authorization URL in the default web browser
webbrowser.open(url)

True

## USER TOP TOKEN

In [31]:
CODE="AQAKxLalHDT7WnSPf8MMI334qk_rdLQi8C0I-Uhdv6mJITu8c6tkDjGBC_4-ROluwVsmcucARsbAFdtBib4G2KU9eR9wq5seKSPRowPaRRdugvDsZe04uf_gB_wena8ouIufJ8nDgG6nsE31vhz9eQbPICBn07Aot9ssEN8n0ZojYLfMPFPhHPE"

### USER TOP SCOPE TOKEN

In [32]:
def get_token_info():
    auth_header =base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    url ="https://accounts.spotify.com/api/token"
    headers={
            "Authorization": f"Basic {auth_header}",
            "Content-Type": "application/x-www-form-urlencoded",}
    data={
            "grant_type": "authorization_code",
            "code" : CODE,
            "redirect_uri": REDIRECT_URI}
    result=requests.post(url, headers=headers, data=data)
    json_results=json.loads(result.content)
    return json_results

token_details=get_token_info()

In [33]:
USER_TOP_ITEM_SCOPE_TOKEN=token_details["access_token"]

In [34]:
def get_top_user_items(url_items):
    response = requests.get(url=url_items, headers={"Authorization" : f"{get_auth_header(USER_TOP_ITEM_SCOPE_TOKEN)}"})
    resp_json = response.json()
    return resp_json

In [39]:
url_items_long = "https://api.spotify.com/v1/me/top/artists?limit=50&offset=0&time_range=long_term"
url_items_short = "https://api.spotify.com/v1/me/top/artists?limit=50&offset=0&time_range=short_term"
results_top_user_items_short = get_top_user_items(url_items_short)
results_top_user_items_long = get_top_user_items(url_items_long)

In [276]:
# results_top_user_items_long
results_top_user_items_long_dict = {i["name"]:i["genres"] for i in results_top_user_items_long['items']}
results_top_user_items_long_dict

{'Juice WRLD': ['chicago rap', 'melodic rap', 'rap'],
 'Trippie Redd': ['melodic rap', 'rap', 'trap'],
 'Kanye West': ['chicago rap', 'hip hop', 'rap'],
 'The Kid LAROI': ['australian hip hop'],
 'Chelsea Cutler': ['alt z'],
 'Drake': ['canadian hip hop', 'canadian pop', 'hip hop', 'pop rap', 'rap'],
 'iann dior': ['melodic rap'],
 'ILLENIUM': ['edm', 'melodic dubstep', 'pop dance'],
 'shinigami': ['cloud rap', 'emo rap', 'scream rap'],
 'Future': ['atl hip hop', 'hip hop', 'rap', 'southern hip hop', 'trap'],
 'Trevor Daniel': ['melodic rap'],
 'Playboi Carti': ['atl hip hop', 'plugg', 'pluggnb', 'rage rap', 'rap'],
 'Polo G': ['chicago rap', 'rap'],
 'NURKO': ['melodic dubstep', 'pop edm'],
 'Pop Smoke': ['brooklyn drill', 'rap'],
 'Machine Gun Kelly': ['ohio hip hop', 'pop rap'],
 'Said The Sky': ['melodic dubstep', 'pop edm'],
 'MOD SUN': ['indie pop rap', 'minnesota hip hop'],
 'Lil Uzi Vert': ['hip hop', 'melodic rap', 'philly rap', 'rage rap', 'rap'],
 'Fivio Foreign': ['brooklyn

In [277]:
# results_top_user_items_long
results_top_user_items_short_dict = {i["name"]:i["genres"] for i in results_top_user_items_short['items']}
results_top_user_items_short_dict

{'Luke Combs': ['contemporary country', 'country'],
 'Caroline Polachek': ['art pop', 'escape room', 'indie pop', 'indietronica'],
 'Chelsea Cutler': ['alt z'],
 'Juice WRLD': ['chicago rap', 'melodic rap', 'rap'],
 'Trippie Redd': ['melodic rap', 'rap', 'trap'],
 'Drake': ['canadian hip hop', 'canadian pop', 'hip hop', 'pop rap', 'rap'],
 'Tate McRae': ['alt z', 'pop'],
 'Future': ['atl hip hop', 'hip hop', 'rap', 'southern hip hop', 'trap'],
 'iann dior': ['melodic rap'],
 'Tory Lanez': ['canadian hip hop',
  'canadian trap',
  'melodic rap',
  'pop rap',
  'r&b',
  'rap',
  'trap'],
 'Lil Uzi Vert': ['hip hop', 'melodic rap', 'philly rap', 'rage rap', 'rap'],
 'NURKO': ['melodic dubstep', 'pop edm'],
 'XXXTENTACION': ['emo rap', 'miami hip hop'],
 'Post Malone': ['dfw rap', 'melodic rap', 'pop', 'rap'],
 'Kendrick Lamar': ['conscious hip hop', 'hip hop', 'rap', 'west coast rap'],
 'A Boogie Wit da Hoodie': ['melodic rap', 'rap', 'trap'],
 'Kanye West': ['chicago rap', 'hip hop', 'ra

In [278]:
from collections import Counter

In [281]:
# def genre_metrics(dictionary):
genre_sublists = results_top_user_items_short_dict.values()
genre_list = [genre for sublist in genre_sublists for genre in sublist]
# distinct_genres = len(set(genre_list))
total_genres = len(genre_list)
temp = Counter(genre_list)
final_percs = {key:round((value/total_genres)*100,2) for key, value in temp.items()}
final = dict(sorted(final_percs.items(), key=lambda x:x[1],  reverse=True))
top_5_genres = [i for i in final.keys()][:10]
top_5_dict = {k: final[k] for k in top_5_genres}
other_dict_temp = {k: final[k] for k in final if k not in top_5_genres}
other_dict = {"other": round(sum(other_dict_temp.values()),2) }
top_5_dict.update(other_dict)
    # return top_5_dict

In [284]:
temp

Counter({'contemporary country': 1,
         'country': 1,
         'art pop': 1,
         'escape room': 1,
         'indie pop': 1,
         'indietronica': 1,
         'alt z': 2,
         'chicago rap': 4,
         'melodic rap': 13,
         'rap': 28,
         'trap': 14,
         'canadian hip hop': 3,
         'canadian pop': 2,
         'hip hop': 12,
         'pop rap': 9,
         'pop': 3,
         'atl hip hop': 7,
         'southern hip hop': 4,
         'canadian trap': 2,
         'r&b': 2,
         'philly rap': 2,
         'rage rap': 2,
         'melodic dubstep': 2,
         'pop edm': 1,
         'emo rap': 2,
         'miami hip hop': 1,
         'dfw rap': 1,
         'conscious hip hop': 3,
         'west coast rap': 1,
         'lgbtq+ hip hop': 1,
         'neo soul': 1,
         'indie pop rap': 4,
         'edm': 2,
         'pop dance': 2,
         'memphis hip hop': 1,
         'cloud rap': 1,
         'scream rap': 1,
         'complextro': 1,
         'e

In [285]:
long_df = pd.DataFrame(top_5_dict, index=[0])
# long_df.insert(0,  "length", "long_term")
long_df

,rap,trap,melodic rap,hip hop,pop rap,atl hip hop,chicago rap,southern hip hop,indie pop rap,canadian hip hop,other
0,17.07,8.54,7.93,7.32,5.49,4.27,2.44,2.44,2.44,1.83,40.26


In [269]:
short_df = pd.DataFrame(genre_metrics(results_top_user_items_short_dict), index=[0])
# short_df.insert(0,  "length", "short_term")
short_df

,rap,trap,melodic rap,hip hop,pop rap,atl hip hop,chicago rap,southern hip hop,indie pop rap,canadian hip hop,other
0,17.07,8.54,7.93,7.32,5.49,4.27,2.44,2.44,2.44,1.83,40.26


In [270]:
combined_df = pd.concat([short_df, long_df], axis=0, join="outer").reset_index().drop(columns=["index"])
combined_df

,rap,trap,melodic rap,hip hop,pop rap,atl hip hop,chicago rap,southern hip hop,indie pop rap,canadian hip hop,other,edm,pop dance,alt z
0,17.07,8.54,7.93,7.32,5.49,4.27,2.44,2.44,2.44,1.83,40.26,NaN,NaN,NaN
1,15.22,6.52,10.14,6.52,5.07,NaN,2.17,NaN,2.90,NaN,43.34,2.9,2.9,2.17


In [271]:
# import plotly.express as px
# fig = px.treemap(combined_df, path=[combined_df.columns], values=combined_df.select_dtypes(include=['float64', 'int64']).values.flatten())
# fig.update_traces(root_color="lightgrey")
# fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
# fig.show()

In [272]:
import plotly.express as px
# fig = px.treemap(short_df, path=[short_df.columns], values=short_df.values.flatten())
fig = px.treemap(short_df, path=[short_df.columns], values=short_df.select_dtypes(include=['float64', 'int64']).values.flatten())
fig.update_traces(root_color="lightgrey", marker=dict(cornerradius=5))
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [273]:
fig = px.treemap(long_df, path=[long_df.columns], values=long_df.select_dtypes(include=['float64', 'int64']).values.flatten())
fig.update_traces(root_color="lightgrey", marker=dict(cornerradius=5))
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [275]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    cols = 2, rows = 1,
    column_widths = [0.4, 0.4],
    subplot_titles = ("Short term Artist Genre Stats" ,"Long term Artist Genre Stats"),
    specs = [[{'type': 'treemap', 'rowspan': 1}, {'type': 'treemap'}]]
)

fig.add_trace(go.Treemap(
    labels = short_df.columns,
    parents=["Short", "Short", "Short", "Short", "Short", "Short", "Short", "Short", "Short", "Short", "Short"],
    values = short_df.values.flatten(),
    textinfo = "label+value",
    root_color="lightgrey"
),row = 1, col = 1)

fig.add_trace(go.Treemap(
    labels = long_df.columns,
    parents=["Long", "Long", "Long", "Long", "Long", "Long", "Long", "Long", "Long", "Long", "Long"],
    values = long_df.values.flatten(),
    textinfo = "label+value",
    root_color="lightgrey"
),row = 1, col = 2)

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [260]:
short_df.columns

Index(['rap', 'trap', 'melodic rap', 'hip hop', 'pop rap', 'other'], dtype='object')

In [261]:
short_df.values

array([[17.07,  8.54,  7.93,  7.32,  5.49, 53.68]])